In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(50000, 32, 32, 3), dtype=float64, chunksize=(10000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(50000, 1), dtype=float64, chunksize=(10000, 1)>,
 dask.array<from-npy-stack, shape=(10000, 32, 32, 3), dtype=float64, chunksize=(1000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(10000, 1), dtype=float64, chunksize=(1000, 1)>)

In [3]:
class CIFAR10Sequence(tf.keras.utils.Sequence):

    def __init__(self, x, y, batch_size):
        self.x, self.y = x, y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

class CyclicLR(tf.keras.callbacks.Callback): 
    '''
    Implementing cyclic learning rate: https://www.computer.org/csdl/proceedings/wacv/2017/4822/00/07926641.pdf
    only triangular mode is considered
    '''
    def __init__(self, min_lr, max_lr, stepsize):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.stepsize = stepsize
        self.iteration = 0
        
    def update_lr(self): #update learning rate after each batch
        cycle = np.floor(1 + self.iteration / (2 * self.stepsize))
        x = np.abs((self.iteration / self.stepsize) - 2 * cycle + 1)
        return self.min_lr + (self.max_lr - self.min_lr) * np.maximum(0, 1 - x)
    
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        self.iteration += 1
        tf.keras.backend.set_value(self.model.optimizer.lr, self.update_lr())
        
class LRTensorBoard(tf.keras.callbacks.TensorBoard):
    '''
    Add learning rate evolution to Tensorboard
    '''
    def __init__(self, log_dir):
        super().__init__(log_dir=log_dir)

    def on_epoch_end(self, epoch, logs=None):
        logs['lr'] = tf.keras.backend.eval(self.model.optimizer.lr)
        super().on_epoch_end(epoch, logs)
        
def build_block(input_layer, filters, norm=True, k=[3,3]):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=(k[0], k[1]), padding='same', use_bias=not norm, kernel_initializer='glorot_uniform')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_model(num_class):
    image_input = tf.keras.Input(shape=(32, 32, 3), name='input_layer')
    conv_1 = build_block(image_input, 48)
    conv_2 = build_block(conv_1, 48)
    pool_1 = tf.keras.layers.MaxPooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.Dropout(0.6)(pool_1)
    conv_3 = build_block(drop_1, 96)
    conv_4 = build_block(conv_3, 96)
    pool_2 = tf.keras.layers.MaxPooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.Dropout(0.6)(pool_2)
    conv_5 = build_block(drop_2, 192)
    conv_6 = build_block(conv_5, 192)
    pool_3 = tf.keras.layers.MaxPooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.Dropout(0.6)(pool_3)
    conv_7 = build_block(drop_3, 192, False, [1,1])
    drop_4 = tf.keras.layers.Dropout(0.25)(conv_7)
    gap = tf.keras.layers.GlobalAvgPool2D()(drop_4)
    logits = tf.keras.layers.Dense(units=num_class, activation='softmax', bias_initializer='ones', kernel_initializer='glorot_uniform')(gap)
    model = tf.keras.Model(inputs=image_input, outputs=logits)
    return model

In [4]:
model = build_model(10)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 48)        1296      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 48)        192       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 48)        20736     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 48)        192       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 48)        0         
__________

In [ ]:
model.load_weights("day_1.weights.best.hdf5")
scores = model.evaluate(test_x, test_y)
scores

In [ ]:
epochs, min_lr, max_lr, batch_size, stepsize = 10000, 0.002, 0.06, 256, 2940
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))
filepath="day_1.weights.best.hdf5"

train_generator = CIFAR10Sequence(train_x, train_y, batch_size)
test_generator = CIFAR10Sequence(test_x, test_y, batch_size)
cyclic_lr = CyclicLR(min_lr=min_lr, max_lr=max_lr, stepsize=stepsize)
tblr = LRTensorBoard(log_dir=os.getcwd())
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

sgd = tf.keras.optimizers.SGD(lr=min_lr)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#model.load_weights(filepath)
model.fit_generator(generator=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=True,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=461,
                    callbacks=[checkpoint, cyclic_lr, tblr])


Epoch 1/10000
195/196 [============================>.] - ETA: 0s - loss: 2.2230 - acc: 0.1831
Epoch 00001: val_acc improved from -inf to 0.09990, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 92s 471ms/step - loss: 2.2220 - acc: 0.1833 - val_loss: 2.9397 - val_acc: 0.0999
Epoch 2/10000
195/196 [============================>.] - ETA: 0s - loss: 1.9466 - acc: 0.2622
Epoch 00002: val_acc improved from 0.09990 to 0.11670, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 32s 162ms/step - loss: 1.9463 - acc: 0.2623 - val_loss: 3.3607 - val_acc: 0.1167
Epoch 3/10000
195/196 [============================>.] - ETA: 0s - loss: 1.8083 - acc: 0.3090
Epoch 00003: val_acc improved from 0.11670 to 0.16710, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 32s 162ms/step - loss: 1.8083 - acc: 0.3091 - val_loss: 3.2142 - val_acc: 0.1671
Epoch 4/10000
195/196 [============================>.] - ETA: 0s

Epoch 29/10000
195/196 [============================>.] - ETA: 0s - loss: 0.9480 - acc: 0.6606
Epoch 00029: val_acc did not improve from 0.63800
196/196 [==============================] - 33s 170ms/step - loss: 0.9481 - acc: 0.6605 - val_loss: 1.0792 - val_acc: 0.6227
Epoch 30/10000
195/196 [============================>.] - ETA: 0s - loss: 0.9491 - acc: 0.6606
Epoch 00030: val_acc did not improve from 0.63800
196/196 [==============================] - 33s 170ms/step - loss: 0.9493 - acc: 0.6605 - val_loss: 1.0775 - val_acc: 0.6263
Epoch 31/10000
195/196 [============================>.] - ETA: 0s - loss: 0.9440 - acc: 0.6610
Epoch 00031: val_acc did not improve from 0.63800
196/196 [==============================] - 33s 170ms/step - loss: 0.9439 - acc: 0.6611 - val_loss: 1.1320 - val_acc: 0.6103
Epoch 32/10000
195/196 [============================>.] - ETA: 0s - loss: 0.9432 - acc: 0.6627
Epoch 00032: val_acc improved from 0.63800 to 0.63960, saving model to day_1.weights.best.hdf5
196

195/196 [============================>.] - ETA: 0s - loss: 0.7913 - acc: 0.7191
Epoch 00058: val_acc did not improve from 0.70500
196/196 [==============================] - 33s 170ms/step - loss: 0.7917 - acc: 0.7189 - val_loss: 0.8482 - val_acc: 0.7008
Epoch 59/10000
195/196 [============================>.] - ETA: 0s - loss: 0.7901 - acc: 0.7158
Epoch 00059: val_acc improved from 0.70500 to 0.72020, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 33s 171ms/step - loss: 0.7898 - acc: 0.7159 - val_loss: 0.7930 - val_acc: 0.7202
Epoch 60/10000
195/196 [============================>.] - ETA: 0s - loss: 0.7861 - acc: 0.7192
Epoch 00060: val_acc did not improve from 0.72020
196/196 [==============================] - 33s 171ms/step - loss: 0.7861 - acc: 0.7192 - val_loss: 0.8262 - val_acc: 0.7072
Epoch 61/10000
195/196 [============================>.] - ETA: 0s - loss: 0.7797 - acc: 0.7204
Epoch 00061: val_acc did not improve from 0.72020
196/196 [=========

195/196 [============================>.] - ETA: 0s - loss: 0.6811 - acc: 0.7575
Epoch 00087: val_acc did not improve from 0.75570
196/196 [==============================] - 33s 171ms/step - loss: 0.6815 - acc: 0.7574 - val_loss: 0.7226 - val_acc: 0.7471
Epoch 88/10000
195/196 [============================>.] - ETA: 0s - loss: 0.6802 - acc: 0.7596
Epoch 00088: val_acc did not improve from 0.75570
196/196 [==============================] - 33s 171ms/step - loss: 0.6796 - acc: 0.7599 - val_loss: 0.6982 - val_acc: 0.7549
Epoch 89/10000
195/196 [============================>.] - ETA: 0s - loss: 0.6703 - acc: 0.7604
Epoch 00089: val_acc improved from 0.75570 to 0.75740, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 34s 171ms/step - loss: 0.6702 - acc: 0.7605 - val_loss: 0.6906 - val_acc: 0.7574
Epoch 90/10000
195/196 [============================>.] - ETA: 0s - loss: 0.6660 - acc: 0.7633
Epoch 00090: val_acc did not improve from 0.75740
196/196 [=========

Epoch 117/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5982 - acc: 0.7876
Epoch 00117: val_acc improved from 0.78130 to 0.78730, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 33s 170ms/step - loss: 0.5979 - acc: 0.7876 - val_loss: 0.6034 - val_acc: 0.7873
Epoch 118/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5919 - acc: 0.7912
Epoch 00118: val_acc did not improve from 0.78730
196/196 [==============================] - 33s 171ms/step - loss: 0.5922 - acc: 0.7910 - val_loss: 0.6125 - val_acc: 0.7865
Epoch 119/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5831 - acc: 0.7955
Epoch 00119: val_acc did not improve from 0.78730
196/196 [==============================] - 33s 170ms/step - loss: 0.5832 - acc: 0.7954 - val_loss: 0.6148 - val_acc: 0.7828
Epoch 120/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5825 - acc: 0.7946
Epoch 00120: val_acc improved from 0.78730 to 0.7

...

195/196 [============================>.] - ETA: 0s - loss: 0.1506 - acc: 0.9442
Epoch 01084: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step - loss: 0.1507 - acc: 0.9442 - val_loss: 0.4202 - val_acc: 0.8863
Epoch 1085/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1526 - acc: 0.9452
Epoch 01085: val_acc did not improve from 0.89160
196/196 [==============================] - 34s 171ms/step - loss: 0.1526 - acc: 0.9452 - val_loss: 0.4249 - val_acc: 0.8849
Epoch 1086/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1552 - acc: 0.9425
Epoch 01086: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step - loss: 0.1554 - acc: 0.9426 - val_loss: 0.4350 - val_acc: 0.8821
Epoch 1087/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1570 - acc: 0.9424
Epoch 01087: val_acc did not improve from 0.89160
196/196 [==============================] - 34s 171ms/step

195/196 [============================>.] - ETA: 0s - loss: 0.1463 - acc: 0.9478
Epoch 01114: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step - loss: 0.1463 - acc: 0.9478 - val_loss: 0.4370 - val_acc: 0.8830
Epoch 1115/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1480 - acc: 0.9469
Epoch 01115: val_acc did not improve from 0.89160
196/196 [==============================] - 34s 171ms/step - loss: 0.1481 - acc: 0.9469 - val_loss: 0.4143 - val_acc: 0.8867
Epoch 1116/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1487 - acc: 0.9464
Epoch 01116: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step - loss: 0.1488 - acc: 0.9464 - val_loss: 0.4499 - val_acc: 0.8795
Epoch 1117/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1539 - acc: 0.9445
Epoch 01117: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step

195/196 [============================>.] - ETA: 0s - loss: 0.1420 - acc: 0.9489
Epoch 01144: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 169ms/step - loss: 0.1421 - acc: 0.9489 - val_loss: 0.4120 - val_acc: 0.8880
Epoch 1145/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1482 - acc: 0.9465
Epoch 01145: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step - loss: 0.1482 - acc: 0.9465 - val_loss: 0.4104 - val_acc: 0.8896
Epoch 1146/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1506 - acc: 0.9454
Epoch 01146: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 171ms/step - loss: 0.1506 - acc: 0.9454 - val_loss: 0.4261 - val_acc: 0.8863
Epoch 1147/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1481 - acc: 0.9459
Epoch 01147: val_acc did not improve from 0.89160
196/196 [==============================] - 33s 170ms/step

195/196 [============================>.] - ETA: 0s - loss: 0.1430 - acc: 0.9479
Epoch 01174: val_acc did not improve from 0.89160
196/196 [==============================] - 36s 181ms/step - loss: 0.1427 - acc: 0.9481 - val_loss: 0.4125 - val_acc: 0.8861
Epoch 1175/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1466 - acc: 0.9462
Epoch 01175: val_acc did not improve from 0.89160
196/196 [==============================] - 34s 174ms/step - loss: 0.1465 - acc: 0.9463 - val_loss: 0.4157 - val_acc: 0.8877
Epoch 1176/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1477 - acc: 0.9460
Epoch 01176: val_acc did not improve from 0.89160
196/196 [==============================] - 34s 172ms/step - loss: 0.1476 - acc: 0.9460 - val_loss: 0.4418 - val_acc: 0.8835
Epoch 1177/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1480 - acc: 0.9471
Epoch 01177: val_acc did not improve from 0.89160
196/196 [==============================] - 35s 177ms/step

Epoch 1204/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1410 - acc: 0.9486
Epoch 01204: val_acc did not improve from 0.89320
196/196 [==============================] - 34s 176ms/step - loss: 0.1410 - acc: 0.9487 - val_loss: 0.4134 - val_acc: 0.8904
Epoch 1205/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1443 - acc: 0.9479
Epoch 01205: val_acc did not improve from 0.89320
196/196 [==============================] - 34s 175ms/step - loss: 0.1444 - acc: 0.9479 - val_loss: 0.4224 - val_acc: 0.8870
Epoch 1206/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1409 - acc: 0.9491
Epoch 01206: val_acc did not improve from 0.89320
196/196 [==============================] - 34s 176ms/step - loss: 0.1409 - acc: 0.9492 - val_loss: 0.4580 - val_acc: 0.8800
Epoch 1207/10000
195/196 [============================>.] - ETA: 0s - loss: 0.1465 - acc: 0.9457
Epoch 01207: val_acc did not improve from 0.89320
196/196 [==============================]